<a href="https://colab.research.google.com/github/watschinger/DATA_SCIENCE_AS_A_FIELD/blob/main/Crypto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [73]:
!pip install talib-binary
!pip install ta
!pip install pandas_ta
from ta import add_all_ta_features
from ta.utils import dropna
import pandas as pd
!pip install yfinance
import yfinance as yf
print('done')

done


In [74]:
cryptotickers = ['BTC-USD', 'ETH-USD', 'USDT-USD', 'BNB-USD', 'USDC-USD', 'XRP-USD', 'ADA-USD']
cryptos = yf.download(cryptotickers, start="2020-01-01", end="2022-02-15", interval="1d", group_by='tickers')
crypto_stacked = cryptos.stack(0).reset_index().rename(columns={'level_1':'Ticker'})
crypto_stacked = crypto_stacked.sort_values(['Ticker','Date']).reset_index(drop=True)
crypto_stacked['Pct_change'] = crypto_stacked.groupby('Ticker')['Close'].pct_change()


[*********************100%***********************]  7 of 7 completed


In [75]:
crypto_stacked

,Date,Ticker,Adj Close,Close,High,Low,Open,Volume,Pct_change
0,2020-01-01,ADA-USD,0.033458,0.033458,0.033813,0.032704,0.032832,22948374,NaN
1,2020-01-02,ADA-USD,0.032751,0.032751,0.033507,0.032448,0.033474,20843934,-0.021131
2,2020-01-03,ADA-USD,0.034180,0.034180,0.034427,0.032491,0.032748,30162644,0.043632
3,2020-01-04,ADA-USD,0.034595,0.034595,0.034685,0.033872,0.034191,29535781,0.012142
4,2020-01-05,ADA-USD,0.034721,0.034721,0.035356,0.034545,0.034574,21479178,0.003642
...,...,...,...,...,...,...,...,...,...
5434,2022-02-11,XRP-USD,0.760226,0.760226,0.829941,0.754331,0.829941,3346204748,-0.084076
5435,2022-02-12,XRP-USD,0.822408,0.822408,0.848838,0.753024,0.760250,3457258389,0.081794
5436,2022-02-13,XRP-USD,0.810273,0.810273,0.844300,0.796277,0.822233,2504198747,-0.014755
5437,2022-02-14,XRP-USD,0.802218,0.802218,0.810474,0.782551,0.809873,2401300497,-0.009941


In [76]:
df = add_all_ta_features(
    crypto_stacked, open="Open", high="High", low="Low", close="Close", volume="Volume", fillna=True)

/usr/local/lib/python3.7/dist-packages/ta/trend.py:769: RuntimeWarning: invalid value encountered in double_scalars
  dip[idx] = 100 * (self._dip[idx] / value)
/usr/local/lib/python3.7/dist-packages/ta/trend.py:774: RuntimeWarning: invalid value encountered in double_scalars
  din[idx] = 100 * (self._din[idx] / value)


In [77]:
df

,Date,Ticker,Adj Close,Close,High,Low,Open,Volume,Pct_change,volume_adi,volume_obv,volume_cmf,volume_fi,volume_em,volume_sma_em,volume_vpt,volume_vwap,volume_mfi,volume_nvi,volatility_bbm,volatility_bbh,volatility_bbl,volatility_bbw,volatility_bbp,volatility_bbhi,volatility_bbli,volatility_kcc,volatility_kch,volatility_kcl,volatility_kcw,volatility_kcp,volatility_kchi,volatility_kcli,volatility_dcl,volatility_dch,volatility_dcm,volatility_dcw,volatility_dcp,volatility_atr,volatility_ui,...,trend_kst_diff,trend_ichimoku_conv,trend_ichimoku_base,trend_ichimoku_a,trend_ichimoku_b,trend_stc,trend_adx,trend_adx_pos,trend_adx_neg,trend_cci,trend_visual_ichimoku_a,trend_visual_ichimoku_b,trend_aroon_up,trend_aroon_down,trend_aroon_ind,trend_psar_up,trend_psar_down,trend_psar_up_indicator,trend_psar_down_indicator,momentum_rsi,momentum_stoch_rsi,momentum_stoch_rsi_k,momentum_stoch_rsi_d,momentum_tsi,momentum_uo,momentum_stoch,momentum_stoch_signal,momentum_wr,momentum_ao,momentum_roc,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,momentum_pvo,momentum_pvo_signal,momentum_pvo_hist,momentum_kama,others_dr,others_dlr,others_cr
0,2020-01-01,ADA-USD,0.033458,0.033458,0.033813,0.032704,0.032832,22948374,NaN,8.256389e+06,22948374,0.359781,0.000000e+00,0.000000,0.000000e+00,7.972416e+07,0.033325,50.000000,1000.000000,0.033458,0.033458,0.033458,0.000000,0.000000,0.0,0.0,0.033325,0.034434,0.032216,6.655666,0.559963,0.0,0.0,0.032704,0.033813,0.033258,3.314604,0.679890,0.000000,0.000000,...,0.000000,0.033258,0.033258,0.033258,0.033258,0.000000,0.000000,0.000000,0.000000,0.000000,4529.116629,4578.344976,4.0,4.0,0.0,-1.000000,-1.000000,0.0,0.0,100.000000,0.000000,0.000000,0.000000,0.000000,0.000000,67.989049,67.989049,-32.010951,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033458,-99.999264,0.000000,0.000000
1,2020-01-02,ADA-USD,0.032751,0.032751,0.033507,0.032448,0.033474,20843934,-0.021131,-6.598406e+05,2104440,-0.015068,-1.473659e+04,-0.000001,-1.427645e-06,-2.338866e+07,0.033124,0.000000,978.869125,0.033104,0.033811,0.032398,4.271303,0.250000,0.0,0.0,0.033114,0.034197,0.032030,6.547178,0.332796,0.0,0.0,0.032448,0.033813,0.033131,4.123302,0.221978,0.000000,0.000000,...,-0.000039,0.033131,0.033131,0.033131,0.033131,0.000000,0.000000,0.000000,0.000000,-66.666667,4529.116629,4578.344976,4.0,8.0,-4.0,-1.000000,-1.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,-100.000000,13.976026,22.197842,45.093445,-77.802158,0.000000,0.000000,-0.168830,-0.033766,-0.135064,-0.736539,-0.147308,-0.589231,0.033161,-2.113087,-2.135733,-2.113087
2,2020-01-03,ADA-USD,0.034180,0.034180,0.034427,0.032491,0.032748,30162644,0.043632,2.180641e+07,32267084,0.294861,-6.473882e+03,0.000003,8.314317e-07,8.756127e+05,0.033358,59.712155,978.869125,0.033463,0.034630,0.032296,6.973638,0.807252,0.0,0.0,0.033309,0.034677,0.031941,8.214050,0.818429,0.0,0.0,0.032448,0.034427,0.033437,5.913986,0.875190,0.000000,0.000000,...,0.000027,0.033437,0.033437,0.033437,0.033437,0.000000,0.000000,0.000000,0.000000,96.012166,4529.116629,4578.344976,12.0,8.0,4.0,-1.000000,0.033813,0.0,1.0,68.520822,0.000000,0.000000,0.000000,-95.607048,48.538068,87.519036,59.235309,-12.480964,0.000000,0.000000,0.041991,-0.018615,0.060605,1.910514,0.264257,1.646258,0.033637,4.363222,4.270715,2.157936
3,2020-01-04,ADA-USD,0.034595,0.034595,0.034685,0.033872,0.034191,29535781,0.012142,4.480295e+07,61802865,0.432918,-3.797988e+03,0.000002,1.306207e-06,1.674676e+06,0.033651,74.766338,990.754196,0.033746,0.035154,0.032338,8.343984,0.801517,0.0,0.0,0.033578,0.034807,0.032348,7.321848,0.913837,0.0,0.0,0.032448,0.034685,0.033567,6.628932,0.959768,0.000000,0.000000,...,0.000067,0.033567,0.033567,0.033567,0.033567,0.000000,0.000000,0.000000,0.000000,115.843800,4529.116629,4578.344976,16.0,8.0,8.0,0.032491,0.033813,1.0,0.0,74.076301,0.000000,0.000000,0.000000,-90.941583,55.216657,95.976786,68.564554,-4.023214,0.000000,0.000000,0.304627,0.046034,0.258593,3.662833,0.943972,2.718861,0.034076,1.2

In [78]:
#Build panel and create interactive chart

In [79]:
from sklearn import preprocessing
import panel as pn
import altair as alt
from vega_datasets import data
import datetime as dt
from altair import datum
alt.renderers.enable('default')
pn.extension('vega')
# remove max row setting
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [80]:
source = crypto_stacked.copy()
source['Date'] = pd.to_datetime(source['Date'])
source.head()

,Date,Ticker,Adj Close,Close,High,Low,Open,Volume,Pct_change,volume_adi,volume_obv,volume_cmf,volume_fi,volume_em,volume_sma_em,volume_vpt,volume_vwap,volume_mfi,volume_nvi,volatility_bbm,volatility_bbh,volatility_bbl,volatility_bbw,volatility_bbp,volatility_bbhi,volatility_bbli,volatility_kcc,volatility_kch,volatility_kcl,volatility_kcw,volatility_kcp,volatility_kchi,volatility_kcli,volatility_dcl,volatility_dch,volatility_dcm,volatility_dcw,volatility_dcp,volatility_atr,volatility_ui,...,trend_kst_diff,trend_ichimoku_conv,trend_ichimoku_base,trend_ichimoku_a,trend_ichimoku_b,trend_stc,trend_adx,trend_adx_pos,trend_adx_neg,trend_cci,trend_visual_ichimoku_a,trend_visual_ichimoku_b,trend_aroon_up,trend_aroon_down,trend_aroon_ind,trend_psar_up,trend_psar_down,trend_psar_up_indicator,trend_psar_down_indicator,momentum_rsi,momentum_stoch_rsi,momentum_stoch_rsi_k,momentum_stoch_rsi_d,momentum_tsi,momentum_uo,momentum_stoch,momentum_stoch_signal,momentum_wr,momentum_ao,momentum_roc,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,momentum_pvo,momentum_pvo_signal,momentum_pvo_hist,momentum_kama,others_dr,others_dlr,others_cr
0,2020-01-01,ADA-USD,0.033458,0.033458,0.033813,0.032704,0.032832,22948374,NaN,8.256389e+06,22948374,0.359781,0.000000,0.000000,0.000000e+00,7.972416e+07,0.033325,50.000000,1000.000000,0.033458,0.033458,0.033458,0.000000,0.000000,0.0,0.0,0.033325,0.034434,0.032216,6.655666,0.559963,0.0,0.0,0.032704,0.033813,0.033258,3.314604,0.679890,0.0,0.0,...,0.000000,0.033258,0.033258,0.033258,0.033258,0.0,0.0,0.0,0.0,0.000000,4529.116629,4578.344976,4.0,4.0,0.0,-1.000000,-1.000000,0.0,0.0,100.000000,0.0,0.0,0.0,0.000000,0.000000,67.989049,67.989049,-32.010951,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033458,-99.999264,0.000000,0.000000
1,2020-01-02,ADA-USD,0.032751,0.032751,0.033507,0.032448,0.033474,20843934,-0.021131,-6.598406e+05,2104440,-0.015068,-14736.593985,-0.000001,-1.427645e-06,-2.338866e+07,0.033124,0.000000,978.869125,0.033104,0.033811,0.032398,4.271303,0.250000,0.0,0.0,0.033114,0.034197,0.032030,6.547178,0.332796,0.0,0.0,0.032448,0.033813,0.033131,4.123302,0.221978,0.0,0.0,...,-0.000039,0.033131,0.033131,0.033131,0.033131,0.0,0.0,0.0,0.0,-66.666667,4529.116629,4578.344976,4.0,8.0,-4.0,-1.000000,-1.000000,0.0,0.0,0.000000,0.0,0.0,0.0,-100.000000,13.976026,22.197842,45.093445,-77.802158,0.0,0.0,-0.168830,-0.033766,-0.135064,-0.736539,-0.147308,-0.589231,0.033161,-2.113087,-2.135733,-2.113087
2,2020-01-03,ADA-USD,0.034180,0.034180,0.034427,0.032491,0.032748,30162644,0.043632,2.180641e+07,32267084,0.294861,-6473.882228,0.000003,8.314317e-07,8.756127e+05,0.033358,59.712155,978.869125,0.033463,0.034630,0.032296,6.973638,0.807252,0.0,0.0,0.033309,0.034677,0.031941,8.214050,0.818429,0.0,0.0,0.032448,0.034427,0.033437,5.913986,0.875190,0.0,0.0,...,0.000027,0.033437,0.033437,0.033437,0.033437,0.0,0.0,0.0,0.0,96.012166,4529.116629,4578.344976,12.0,8.0,4.0,-1.000000,0.033813,0.0,1.0,68.520822,0.0,0.0,0.0,-95.607048,48.538068,87.519036,59.235309,-12.480964,0.0,0.0,0.041991,-0.018615,0.060605,1.910514,0.264257,1.646258,0.033637,4.363222,4.270715,2.157936
3,2020-01-04,ADA-USD,0.034595,0.034595,0.034685,0.033872,0.034191,29535781,0.012142,4.480295e+07,61802865,0.432918,-3797.987544,0.000002,1.306207e-06,1.674676e+06,0.033651,74.766338,990.754196,0.033746,0.035154,0.032338,8.343984,0.801517,0.0,0.0,0.033578,0.034807,0.032348,7.321848,0.913837,0.0,0.0,0.032448,0.034685,0.033567,6.628932,0.959768,0.0,0.0,...,0.000067,0.033567,0.033567,0.033567,0.033567,0.0,0.0,0.0,0.0,115.843800,4529.116629,4578.344976,16.0,8.0,8.0,0.032491,0.033813,1.0,0.0,74.076301,0.0,0.0,0.0,-90.941583,55.216657,95.976786,68.564554,-4.023214,0.0,0.0,0.304627,0.046034,0.258593,3.662833,0.943972,2.718861,0.034076,1.214163,1.206852,3.398300
4,2020-01-05,ADA-USD,0.034721,0.034721,0.035356,0.034545,0.034574,21479178,0.003642,3.264631e+07,83282043,0.261233,-2868.802607,0.000003,1.613980e-06,4.368409e+05,0.033861,80.218383,994.362574,0.033941,

In [81]:
# Create the multiple ticker chart
def multiple_ticker_chart(df,y_axis_sel):
  #y_axis_sel = 'Close_res'

  # Create a selection that chooses the nearest point & selects based on x-value
  nearest = alt.selection(type='single', nearest=True, on='mouseover',
                        fields=['Date'], empty='none')

  # The basic line for pct change we use scatter plot, for price we use line
  if y_axis_sel == 'Close_res':
    line = alt.Chart().mark_line(interpolate='basis').encode(
        alt.X('Date:T', axis=alt.Axis(title='')),
        alt.Y(y_axis_sel +':Q', axis=alt.Axis(title='',format='%')),
        alt.Tooltip(['Date','Ticker']),
        color='Ticker:N'
    )
  else:
    line = alt.Chart().mark_circle(size = 20).encode(
        alt.X('Date:T', axis=alt.Axis(title='')),
        alt.Y(y_axis_sel +':Q', axis=alt.Axis(title='',format='%')),
        alt.Tooltip(['Date','Ticker']),
        color='Ticker:N'
    )
# Transparent selectors across the chart. This is what tells us
# the x-value of the cursor
  selectors = alt.Chart().mark_point().encode(
      x='Date:T',
      opacity=alt.value(0),
  ).add_selection(
      nearest
  )

  # Draw points on the line, and highlight based on selection
  points = line.mark_point().encode(
      opacity=alt.condition(nearest, alt.value(1), alt.value(0))
  )

  # Draw text labels near the points, and highlight based on selection
  text = line.mark_text(align='left', dx=5, dy=-5).encode(
      text=alt.condition(nearest, y_axis_sel +':Q', alt.value(' '))
  )

  # Draw a rule at the location of the selection
  rules = alt.Chart().mark_rule(color='gray').encode(
      x='Date:T',
  ).transform_filter(
      nearest
  )

  
  # Put the five layers into a chart and bind the data
  cryptoChart = alt.layer(line, selectors, points, rules, text,
                       data=df[['Ticker','Date',y_axis_sel]], 
                       width=700, height=270,title='Crypto History Indexed')
  return cryptoChart


In [82]:
# set a title for your dashboard
title = '## Crypto Technical Indicator Dashboard'

subtitle = 'This dashboard allows you to screen a set of technical indictors and to select a date range to analyse crypto currencies'

# create list of company names (tickers)
tickers = cryptotickers.copy()
tickers.insert(0,'_ALL')

# this creates the dropdown widget
ticker = pn.widgets.Select(name='Cryptocurrencies', options=tickers)

# create a dropdown widget to switch between daily changes and indexed performance
y_axis_main = pn.widgets.Select(name='Y-axis', options=['Daily_change','Indexed'])

# this creates the date range slider
date_range_slider = pn.widgets.DateRangeSlider(
    name='Date Range Slider',
    start=source['Date'].min(), end=source['Date'].max(),
    value=(source['Date'].min(), source['Date'].max())
)

# tell Panel what your plot "depends" on. 
# This defines what should trigger a change in the chart.
# both values in depends() will be used in our below Altair chart as filters
@pn.depends(ticker.param.value, date_range_slider.param.value, y_axis_main.param.value)
def get_plot(ticker, date_range, y_axis_main): # start function
    # Decide what to display on Y axis - daily percentage changes or performance over time
    if y_axis_main != 'Daily_change':
      y_column = 'Close_res'
    else:
      y_column = 'Pct_change'
    # Load and format the data
    df = source.copy() # define df
    df['Date'] = pd.to_datetime(df['Date']) # format date as datetime
    
    # create a date filter that uses values from the date range slider
    start_date = date_range_slider.value[0] # store the first date range slider value in a var
    end_date = date_range_slider.value[1] # store the end date in a var
    mask = (df['Date'] > start_date) & (df['Date'] <= end_date) # create filter mask for the dataframe
    df_selected = df.loc[mask].copy() # filter the dataframe
    
    # Indexing the close prices
    def start_value(order_id):
      #print(order_id)
      min_idx =  df_selected.groupby('Ticker')['Date'].idxmin()
      #print(min_idx)
      #print(df_init.loc[min_idx, ['Date','Ticker','Close']])
      return df_selected.loc[min_idx, ['Ticker','Close']]
    x = start_value(cryptotickers)

    df_selected['Start'] = df_selected['Ticker'].map(x.set_index('Ticker')['Close'])
    df_selected['Close_res'] = df_selected['Close']/df_selected['Start']
    #df_init

    # If all selection in cryptocurrencies is "_ALL" then return the indexed price performance, else a simple price chart
    if ticker == '_ALL':
      chart = multiple_ticker_chart(df_selected,y_column)
    else:
      # create the Altair chart object for single ticker
      if y_axis_main != 'Daily_change':
          chart = alt.Chart(df_selected).mark_area(color="#0c1944", opacity=0.8).encode(x='Date', y='Close', tooltip=alt.Tooltip(['Date','Close'])).transform_filter(
            (datum.Ticker == ticker) # this ties in the filter from the dropdown selection
          ).properties(
            width=700,
            height=270,
            title=ticker
          )
      else:
          chart = alt.Chart(df_selected).mark_circle(size = 20).encode(x='Date',  y='Pct_change', tooltip=alt.Tooltip(['Date','Pct_change'])).transform_filter(
          (datum.Ticker == ticker) # this ties in the filter from the dropdown selection
          ).properties(
            width=700,
            height=270,
            title=ticker
          )
    
    return chart


@pn.depends(ticker.param.value, date_range_slider.param.value, y_axis_main.param.value)
def get_plot2(ticker, date_range, y_axis_main): # start function
    # Decide what to display on Y axis - daily percentage changes or performance over time
    if y_axis_main != 'Daily_change':
      y_column = 'Close_res'
    else:
      y_column = 'Pct_change'
    # Load and format the data
    df = source.copy() # define df
    df['Date'] = pd.to_datetime(df['Date']) # format date as datetime
    
    # create a date filter that uses values from the date range slider
    start_date = date_range_slider.value[0] # store the first date range slider value in a var
    end_date = date_range_slider.value[1] # store the end date in a var
    mask = (df['Date'] > start_date) & (df['Date'] <= end_date) # create filter mask for the dataframe
    df_selected = df.loc[mask].copy() # filter the dataframe
    
    # Indexing the close prices
    def start_value(order_id):
      #print(order_id)
      min_idx =  df_selected.groupby('Ticker')['Date'].idxmin()
      #print(min_idx)
      #print(df_init.loc[min_idx, ['Date','Ticker','Close']])
      return df_selected.loc[min_idx, ['Ticker','Close']]
    x = start_value(cryptotickers)

    df_selected['Start'] = df_selected['Ticker'].map(x.set_index('Ticker')['Close'])
    df_selected['Close_res'] = df_selected['Close']/df_selected['Start']
    #df_init

    
    
    #Create the second chart - correlation for multiple view

    if ticker !='_ALL':
    # Technical Indicators
      df_ti = df[df['Ticker'] == ticker].copy()
      df_ti = df_ti.reset_index()
      df_ti = df_ti[['Date','Pct_change','volume_vwap','volume_mfi','volume_cmf','volatility_bbm','volatility_bbh','volatility_bbl','trend_kst_diff','trend_adx','trend_cci','trend_aroon_ind','trend_psar_up_indicator','trend_psar_down_indicator','momentum_rsi','momentum_stoch_rsi','momentum_tsi','momentum_ppo_signal','momentum_pvo_signal','momentum_kama']].copy()
      df_ti['Pct_change_foll'] = df_ti['Pct_change'].shift(periods=-1)
      df_ti=df_ti.dropna()
      df_to_norm = df_ti.drop(['Date'], axis=1)
      df_to_keep =  df_ti[[ 'Date']]
      col_headers = df_to_norm.columns
      x = df_to_norm.values #returns a numpy array
      #print(df_to_keep)
      min_max_scaler = preprocessing.MinMaxScaler()
      x_scaled = min_max_scaler.fit_transform(x)
      df_to_norm = pd.DataFrame(x_scaled)
      df_to_norm.columns = col_headers
      df_to_norm = df_to_norm.rename(columns={'Pct_change': 'Pct_change_norm', 'Pct_change_foll': 'Pct_change_foll_norm'})
      df_fin= df_to_keep.join(df_to_norm)
      #print(df_fin)
      df_fin=df_fin.set_index("Date").stack().reset_index().rename(columns={"level_1": "Indicator", 0: "normalized"})
      df_fin=df_fin.sort_values(['Date', "Indicator"], ascending = (True, True)).reset_index(drop=True)
      mask = (df_fin['Date'] > start_date) & (df_fin['Date'] <= end_date) # create filter mask for the dataframe
      df_selected = df_fin.loc[mask].copy() # filter the dataframe

      base2 = alt.Chart(df_selected).encode(
        x=alt.X('Date:O', axis=None),
        #x='Date:T',
        y='Indicator:O'    
        ).properties(
        width=700,
        height=400,
        title='Technical Indicators normalized'
        ).configure_axisY(
          titleAngle=0, 
          #titleY=-10,
          #titleX=-60,
          labelPadding=-720, 
          labelAlign='left'
      )

      # The tech indicator heatmap itself
      color_plot = base2.mark_rect().encode(
          color='normalized:Q'
      )

      chart3 = color_plot  

     
    if ticker == '_ALL':
      
      # Correlation matrix and corr chart
      corr_in = df_selected[['Date','Ticker','Pct_change']].copy()
      corr_in = corr_in.pivot(index='Date', columns='Ticker', values='Pct_change').dropna()
      corr_matrix = corr_in.corr()
      corr_matrix = corr_matrix.rename_axis(None).rename_axis(None, axis=1)
      corr_matrix = corr_matrix.stack().reset_index()
      corr_data = corr_matrix.rename(columns={0: 'correlation', 'level_0': 'variable', 'level_1': 'variable2'}).copy()
      corr_data['correlation_label'] = corr_data['correlation'].map('{:.2f}'.format)  # Round to 2 decimal
      corr_data.head()

      base = alt.Chart(corr_data).encode(
          x='variable2:O',
          y='variable:O'    
      ).properties(
      width=600,
      height=150,
      title='Crypto Correlation Matrix'
      )

      # Text layer with correlation labels
      # Colors are for easier readability
      text = base.mark_text().encode(
          text='correlation_label',
          color=alt.condition(
              alt.datum.correlation > 0.5, 
              alt.value('white'),
              alt.value('black')
          )
        )

      # The correlation heatmap itself
      cor_plot = base.mark_rect().encode(
          color='correlation:Q'
      )

      chart3 = cor_plot + text # The '+' means overlaying the text and rect layer
      

     

    return chart3



# create the Panel object, passing in all smaller objects
dashboard = pn.Row(
    pn.Column(title, subtitle, ticker, y_axis_main,date_range_slider),
    pn.Column(get_plot,pn.pane.Markdown('', background='#f0f0f0', width=750, height=20),get_plot2) # draw chart function!
)

In [83]:
dashboard.servable()

Row
    [0] Column
        [0] Markdown(str)
        [1] Markdown(str)
        [2] Select(name='Cryptocurrencies', options=['_ALL', 'BTC-USD', ...], value='_ALL')
        [3] Select(name='Y-axis', options=['Daily_change', ...], value='Daily_change')
        [4] DateRangeSlider(end=Timestamp('2022-02-15 0..., name='Date Range Slider', start=Timestamp('2020-01-01 0..., value=(Timestamp('2020-01-01 00:..., value_end=Timestamp('2022-02-15 0..., value_start=Timestamp('2020-01-01 0...)
    [1] Column
        [0] ParamFunction(function)
        [1] Markdown(str, background='#f0f0f0', height=20, sizing_mode='fixed', width=750)
        [2] ParamFunction(function)